## Interactive Cropping Tool Demo

In [2]:
from interactivecrop.interactivecrop import main as crop
from interactivecrop.samples import sample_images, sample_names

crop(sample_images,sample_names, optimize=False)

GridBox(children=(Dropdown(description='Img Name:', layout=Layout(grid_area='im_selector', width='100%'), opti…